In [10]:
%load_ext autoreload
%autoreload 2

from src.data.skin_cancer import SkinCancerDataset
from src.models.convolutional import ConvVariationalAutoencoder, ConvSphericalVAE
from sklearn.decomposition import PCA
from src.utils import plot_3d
import optuna
import torch

from plotly.subplots import make_subplots

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
vae_study = optuna.load_study(
    study_name="skin-cancer-vae",
    storage="sqlite:///../runs/skin-cancer-vae/optuna-storage.db"
)
best_vae_trial = vae_study.best_trial

svae_study = optuna.load_study(
    study_name="skin-cancer-svae",
    storage="sqlite:///../runs/skin-cancer-svae/optuna-storage.db"
)
best_svae_trial = svae_study.best_trial

In [61]:
dataset = SkinCancerDataset(image_size=[225, 300], test=True)
X = dataset.X
image_size = X.shape[1:]

In [62]:
vae_params = vae_study.best_trial.params
vae_ffnn_layer_sizes = [vae_params[f"ffnn_layers_size_{i+1}"] for i in range(vae_params["n_ffnn_layers"])]
vae_kernel_sizes = [vae_params[f"kernel_size_{i+1}"] for i in range(vae_params["n_conv_layers"])]
vae_maxpool_kernel = vae_params["maxpool_kernel"]
vae_maxpool_stride = vae_params["maxpool_stride"]
vae_out_channels = [vae_params[f"out_channels_{i+1}"] for i in range (vae_params["n_conv_layers"])]
vae_padding_sizes = [vae_params[f"padding_size_{i+1}"] for i in range (vae_params["n_conv_layers"])]
vae_stride = [vae_params[f"stride_{i+1}"] for i in range (vae_params["n_conv_layers"])]
vae_dropout = vae_study.best_trial.params["dropout"]
vae_number = vae_study.best_trial.number

svae_params = svae_study.best_trial.params
svae_ffnn_layer_sizes = [svae_params[f"ffnn_layers_size_{i+1}"] for i in range(svae_params["n_ffnn_layers"])]
svae_kernel_sizes = [svae_params[f"kernel_size_{i+1}"] for i in range(svae_params["n_conv_layers"])]
svae_maxpool_kernel = svae_params["maxpool_kernel"]
svae_maxpool_stride = svae_params["maxpool_stride"]
svae_out_channels = [svae_params[f"out_channels_{i+1}"] for i in range (svae_params["n_conv_layers"])]
svae_padding_sizes = [svae_params[f"padding_size_{i+1}"] for i in range (svae_params["n_conv_layers"])]
svae_stride = [svae_params[f"stride_{i+1}"] for i in range (svae_params["n_conv_layers"])]
svae_dropout = svae_study.best_trial.params["dropout"]
svae_number = svae_study.best_trial.number


In [54]:
image_size

torch.Size([3, 225, 300])

In [63]:
vae = ConvVariationalAutoencoder(
    image_size = image_size,
    latent_dim = 2,
    encoder_params={
        "kernel_size": vae_kernel_sizes,
        "stride" : vae_stride,
        "out_channel_size" : vae_out_channels,
        "padding_size" : vae_padding_sizes,
        "activation_function" : "Tanh",
        "ffnn_layer_size" : vae_ffnn_layer_sizes,
        "dropout" : vae_dropout,
        "dropout2d" : vae_dropout,
        "maxpool_kernel" : vae_maxpool_kernel,
        "maxpool_stride" : vae_maxpool_stride
     },
    decoder_params={
        "kernel_size" : vae_kernel_sizes[::-1],
        "stride" : vae_stride[::-1],
        "in_channel_size" : vae_out_channels[::-1],
        "activation_function" : "Tanh",
        "ffnn_layer_size" : vae_ffnn_layer_sizes[::-1],
        "dropout" : vae_dropout,
        "dropout2d" : vae_dropout
    },
)
vae_state_dict = torch.load(f"../runs/skin-cancer-vae/checkpoints/{vae_number:03}.pt", map_location=torch.device('cpu'))
vae.load_state_dict(vae_state_dict)

<All keys matched successfully>

In [64]:
svae = ConvSphericalVAE(
    image_size = image_size,
    latent_dim = 3,
    encoder_params={
        "kernel_size" : svae_kernel_sizes,
        "stride" : svae_stride,
        "out_channel_size" : svae_out_channels,
        "padding_size" : svae_padding_sizes,
        "activation_function" : "Tanh",
        "ffnn_layer_size" : svae_ffnn_layer_sizes,
        "dropout" : svae_dropout,
        "dropout2d" : svae_dropout,
        "maxpool_kernel" : svae_maxpool_kernel,
        "maxpool_stride" : svae_maxpool_stride
     },
    decoder_params={
        "kernel_size" : svae_kernel_sizes[::-1],
        "stride" : svae_stride[::-1],
        "in_channel_size" : svae_out_channels[::-1],
        "activation_function" : "Tanh",
        "ffnn_layer_size" : svae_ffnn_layer_sizes[::-1],
        "dropout" : svae_dropout,
        "dropout2d" : svae_dropout
    },
)
svae_state_dict = torch.load(f"../runs/skin-cancer-svae/checkpoints/{svae_number:03}.pt", map_location=torch.device('cpu'))
svae.load_state_dict(svae_state_dict)

<All keys matched successfully>

In [60]:
import re
pattern = re.compile("_(\d+):")
classes = [int(pattern.search(s).group(1)) for s in dataset.labels]

AttributeError: 'SkinCancerDataset' object has no attribute 'labels'

In [65]:
fig = make_subplots(
    rows=1,
    cols=3,
    specs=[
        [{"type": "scatter3d"}, {"type": "scatter3d"}, {"type": "scatter3d"}]
    ],
    subplot_titles=(
        "PCA",
        "Ordinary VAE",
        "Spherical VAE",
    ),
    horizontal_spacing=0.01,
    vertical_spacing=0.01,
)

In [77]:
x, y, z = PCA(n_components=3).fit_transform(X[0,0,:,:]).T
plot_3d( x, y, z, fig=fig, row=1, col=1);

In [78]:
output = svae(X)
plot_3d( *output["z"].detach().numpy().T, fig=fig, row=1, col=3);

In [70]:
output = vae(X)
plot_3d( *output["z"].detach().numpy().T, fig=fig, row=1, col=2);

TypeError: plot_3d() missing 1 required positional argument: 'z'

In [79]:
fig